In [25]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import sklearn
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")


# Input file containing data
input_file = 'E:\income_data.txt'

# Read the data
X = []
y = []
count_class1 = 0
count_class2 = 0
max_datapoints = 25000

#open the file and check for the salary less than or greater than 50k
with open(input_file, 'r') as f:
    for line in f.readlines():
        if count_class1 >= max_datapoints and count_class2 >= max_datapoints:
            break

        #if '?' in line:
         #continue

        data = line[:-1].split(', ')

        if data[-1] == '<=50K' and count_class1 < max_datapoints:
            X.append(data)
            count_class1 += 1

        if data[-1] == '>50K' and count_class2 < max_datapoints:
            X.append(data)
            count_class2 += 1

# Convert to numpy array
X = np.array(X)
print(X)

[['39' 'State-gov' '77516' ... '40' 'United-States' '<=50K']
 ['50' 'Self-emp-not-inc' '83311' ... '13' 'United-States' '<=50K']
 ['38' 'Private' '215646' ... '40' 'United-States' '<=50K']
 ...
 ['58' 'Private' '151910' ... '40' 'United-States' '<=50K']
 ['22' 'Private' '201490' ... '20' 'United-States' '<=50K']
 ['52' 'Self-emp-inc' '287927' ... '40' 'United-States' '>50K']]


In [31]:
label_encoder = [] 

#Encode string data to numeric
X_encoded = np.empty(X.shape)
for i,item in enumerate(X[0]):
    if item.isdigit(): 
        X_encoded[:, i] = X[:, i]
    elif item=="?":
        X_encoded[:,i]=109 #replace ? with 109
    else:
        label_encoder.append(preprocessing.LabelEncoder())
        X_encoded[:, i] = label_encoder[-1].fit_transform(X[:, i])


    
X = X_encoded[:, :-1].astype(int)
m=stats.mode(X)
l1=m[0]
print('list 1',l1)
y = X_encoded[:, -1].astype(int)
print(X,y)

#if a ? is found, replace it with the highest occuring value in the column
for i,item in enumerate(X[0]):
    if item==109:
        item[:,i]=l1[i]
    


list 1 [[    36      4 123011     11      9      2     10      0      4      1
       0      0     40     39]]
[[    39      7  77516 ...      0     40     39]
 [    50      6  83311 ...      0     13     39]
 [    38      4 215646 ...      0     40     39]
 ...
 [    58      4 151910 ...      0     40     39]
 [    22      4 201490 ...      0     20     39]
 [    52      5 287927 ...      0     40     39]] [0 0 0 ... 0 0 1]


In [38]:
# SVM CLASSIFIER

import sklearn
classifier = OneVsOneClassifier(LinearSVC(random_state=0))

# Train the classifier
classifier.fit(X, y)

# Cross validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)
classifier = OneVsOneClassifier(LinearSVC(random_state=0))
classifier.fit(X_train, y_train)
y_test_pred = classifier.predict(X_test)


###############################################
# Scoring functions

num_folds = 3
accuracy_values = sklearn.model_selection.cross_val_score(classifier, X, y, scoring='accuracy', cv=num_folds)
print("Accuracy: " + str(round(100*accuracy_values.mean(), 2)) + "%")

precision_values = sklearn.model_selection.cross_val_score(classifier, X, y, scoring='precision_weighted', cv=num_folds)
print("Precision: " + str(round(100*precision_values.mean(), 2)) + "%")

recall_values = sklearn.model_selection.cross_val_score(classifier, X, y, scoring='recall_weighted', cv=num_folds)
print("Recall: " + str(round(100*recall_values.mean(), 2)) + "%")

# Compute the F1 score of the SVM classifier
f1 = cross_val_score(classifier, X, y, scoring='f1_weighted', cv=3)
print("F1 score: " + str(round(100*f1.mean(), 2)) + "%")




Accuracy: 78.62%
Precision: 79.8%
Recall: 78.62%
F1 score: 73.67%


In [37]:
#LOGISTIC REGRESSION

LR=LogisticRegression(random_state=0,solver='lbfgs',multi_class='ovr').fit(X_train,y_train)
y_test_pred=LR.predict(X_test)

# Scoring functions

num_folds = 3
accuracy_values = sklearn.model_selection.cross_val_score(LR, X, y, scoring='accuracy', cv=num_folds)
print("Accuracy: " + str(round(100*accuracy_values.mean(), 2)) + "%")

precision_values = sklearn.model_selection.cross_val_score(LR, X, y, scoring='precision_weighted', cv=num_folds)
print("Precision: " + str(round(100*precision_values.mean(), 2)) + "%")

recall_values = sklearn.model_selection.cross_val_score(LR, X, y, scoring='recall_weighted', cv=num_folds)
print("Recall: " + str(round(100*recall_values.mean(), 2)) + "%")

# Compute the F1 score of the SVM classifier
f1 = cross_val_score(LR, X, y, scoring='f1_weighted', cv=3)
print("F1 score: " + str(round(100*f1.mean(), 2)) + "%")




Accuracy: 79.2%
Precision: 77.32%
Recall: 79.2%
F1 score: 75.92%


In [36]:
#RANDOM FOREST CLASSIFIER

RF=RandomForestClassifier(random_state=0)
RF.fit(X_train,y_train)
y_test_pred=RF.predict(X_test)

# Scoring functions

num_folds = 3
accuracy_values = sklearn.model_selection.cross_val_score(RF, X, y, scoring='accuracy', cv=num_folds)
print("Accuracy: " + str(round(100*accuracy_values.mean(), 2)) + "%")

precision_values = sklearn.model_selection.cross_val_score(RF, X, y, scoring='precision_weighted', cv=num_folds)
print("Precision: " + str(round(100*precision_values.mean(), 2)) + "%")

recall_values = sklearn.model_selection.cross_val_score(RF, X, y, scoring='recall_weighted', cv=num_folds)
print("Recall: " + str(round(100*recall_values.mean(), 2)) + "%")


f1 = cross_val_score(RF, X, y, scoring='f1_weighted', cv=3)
print("F1 score: " + str(round(100*f1.mean(), 2)) + "%")

Accuracy: 84.98%
Precision: 84.29%
Recall: 84.98%
F1 score: 84.36%


In [35]:
#NAIVE BAYES CLASSIFIER

class2=GaussianNB()
class2.fit(X_train,y_train)
y_test_pred=class2.predict(X_test)

num_folds = 3
accuracy_values = sklearn.model_selection.cross_val_score(class2, X, y, scoring='accuracy', cv=num_folds)
print("Accuracy: " + str(round(100*accuracy_values.mean(), 2)) + "%")

precision_values = sklearn.model_selection.cross_val_score(class2, X, y, scoring='precision_weighted', cv=num_folds)
print("Precision: " + str(round(100*precision_values.mean(), 2)) + "%")

recall_values = sklearn.model_selection.cross_val_score(class2, X, y, scoring='recall_weighted', cv=num_folds)
print("Recall: " + str(round(100*recall_values.mean(), 2)) + "%")


f1 = cross_val_score(class2, X, y, scoring='f1_weighted', cv=3)
print("F1 score: " + str(round(100*f1.mean(), 2)) + "%")

Accuracy: 79.52%
Precision: 77.57%
Recall: 79.52%
F1 score: 76.63%
